In [19]:
import util
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as ta
import warnings
import numpy as np

def warn(*args, **kwargs):
    pass
warnings.warn = warn
np.seterr(divide='ignore')

def indicador(df_campo):
    indi=pd.Series(df_campo)
    return indi.to_numpy()

class IndiCross(Strategy):
    def init(self):
        data['columna1'] = ta.ema(data.Close, length=100)
        data['columna2'] = ta.ema(data.Close, length=200)
        self.columna1 = self.I(indicador, data.columna1,name="columna1", color="red")
        self.columna2 = self.I(indicador, data.columna2,name="columna2", color="blue")
    def next(self):
        if crossover(self.columna1, self.columna2):
            self.buy()
        elif crossover(self.columna2, self.columna1):
            self.sell()

##################                

lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'DOGEUSDT', 'DOTUSDT', 
         'SOLUSDT', 'AVAXUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'OPUSDT', 'FETUSDT', 'AGIXUSDT', 'ARBUSDT',  
         'SLPUSDT', 'MEMEUSDT',  '1000SATSUSDT', 'PIXELUSDT']
lista=['XRPUSDT']
timeframe='1h'
suma=0.0
for symbol in lista:
    data=util.obtiene_historial(symbol,timeframe)
    bt = Backtest(data,IndiCross,cash=100000,exclusive_orders=True)
    output = bt.run()
    bt.plot()
    print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)}")
    suma=suma+output['Return [%]']
suma

Symbol: XRPUSDT - Return [%]: -18.09


-18.089661899999978

In [20]:
data

,Open Time,Open,High,Low,Close,Volume,ema20,ema50,ema200,sma200,atr,columna1,columna2
2024-07-07 23:00:00,2024-07-07 23:00:00,0.4203,0.4232,0.4179,0.4194,29314956.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-08 00:00:00,2024-07-08 00:00:00,0.4194,0.4204,0.4088,0.4105,97784141.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-08 01:00:00,2024-07-08 01:00:00,0.4105,0.4110,0.4028,0.4102,115485966.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-08 02:00:00,2024-07-08 02:00:00,0.4103,0.4164,0.4090,0.4148,63346494.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-08 03:00:00,2024-07-08 03:00:00,0.4148,0.4167,0.4121,0.4143,37847269.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-18 10:00:00,2024-08-18 10:00:00,0.5667,0.5705,0.5663,0.5668,33623066.9,0.565918,0.565703,0.567875,0.571187,0.002699,0.566713,0.567875
2024-08-18 11:00:00,2024-08-18 11:00:00,0.5668,0.5677,0.5659,0.5677,11435286.2,0.566088,0.565781,0.567874,0.571080,0.002635,0.566733,0.567874
2024-08-18 12:00:00,2024-08-18 12:00:00,0.5676,0.5844,0.5671,0.5733,126948211.3,0.566774,0.566076,0.567928,0.571011,0.003683,0.566863,0.567928
2024-08-18 13:00:00,2024-08-18 13:00:00,0.5733,0.5746,0.5697,0.5712,49730947.2,0.567196,0.566277,0.567960,0.570945,0.003769,0.566949,0.567960
